## Imports

In [38]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [39]:
train = pd.read_csv('train.csv')
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,1
1,4,NaN,NaN,forest fire near la rong sask canada,1
2,5,NaN,NaN,all resid ask to ishelt in place are be notifi...,1
3,6,NaN,NaN,13000 peopl receiv wildfir evacu order in cali...,1
4,7,NaN,NaN,just got sent thi photo from rubi alaska as sm...,1
5,8,NaN,NaN,rockyfir updat california hwi 20 close in both...,1
6,10,NaN,NaN,flood disast heavi rain cau flash flood of str...,1
7,13,NaN,NaN,i m on top of the hill and i can see a fire in...,1
8,14,NaN,NaN,there i an emerg evacu happen now in the build...,1
9,15,NaN,NaN,i m afraid that the tornado is come to our area,1


In [40]:
pos_col_text = train.columns.get_loc('text')+1
train.insert(loc=pos_col_text, column='len_text', value=train['text'].iloc[:].str.len())
train.head(10)

,id,keyword,location,text,len_text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,65,1
1,4,NaN,NaN,forest fire near la rong sask canada,37,1
2,5,NaN,NaN,all resid ask to ishelt in place are be notifi...,108,1
3,6,NaN,NaN,13000 peopl receiv wildfir evacu order in cali...,53,1
4,7,NaN,NaN,just got sent thi photo from rubi alaska as sm...,82,1
5,8,NaN,NaN,rockyfir updat california hwi 20 close in both...,92,1
6,10,NaN,NaN,flood disast heavi rain cau flash flood of str...,80,1
7,13,NaN,NaN,i m on top of the hill and i can see a fire in...,56,1
8,14,NaN,NaN,there i an emerg evacu happen now in the build...,65,1
9,15,NaN,NaN,i m afraid that the tornado is come to our area,48,1


### Enconding de columnas categóricas/strings

In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk #pip install nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ben/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False, ngram_range=(1,1), stop_words='english', tokenizer=nltk.word_tokenize)
tf_idf_data = tf_idf_vec.fit_transform(train['text'])

In [43]:
col_names = tf_idf_vec.get_feature_names()
col_names

['0',
 '00',
 '01',
 '02',
 '0215',
 '03',
 '030',
 '034',
 '039',
 '05',
 '05th',
 '06',
 '060',
 '061',
 '06jst',
 '07',
 '08',
 '0840728',
 '09',
 '0npzp',
 '1',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000and',
 '1008pla',
 '1008planet',
 '100bn',
 '100mb',
 '100nd',
 '101',
 '1017',
 '1023',
 '1028',
 '103',
 '105',
 '106',
 '1061thetwist',
 '107',
 '109',
 '10am',
 '10k',
 '10km',
 '10m',
 '10new',
 '10pm',
 '10th',
 '10w',
 '10x',
 '11',
 '11000',
 '11000000',
 '111020',
 '114',
 '1141',
 '1145',
 '115',
 '115film',
 '119000',
 '11am',
 '12',
 '1200',
 '12000',
 '120000',
 '1200000',
 '1236',
 '124',
 '125',
 '129',
 '12am',
 '12hr',
 '12jst',
 '12m',
 '12mm',
 '12new',
 '12pm',
 '12th',
 '12u',
 '12v',
 '13',
 '130',
 '13000',
 '133',
 '138',
 '139055',
 '13pm',
 '14',
 '140',
 '14000',
 '14028',
 '141',
 '143',
 '148',
 '149',
 '14th',
 '15',
 '150',
 '1500',
 '15000270364',
 '15000270653',
 '158',
 '159',
 '15901',
 '15km',
 '15moferadio',
 '15p',
 '15pm',
 '15t',
 '15

In [44]:
tf_idf_array = tf_idf_data.toarray()

In [45]:
tf_idf_df = pd.DataFrame(tf_idf_array, columns=col_names)
train_set_encoded = train.join(other=tf_idf_df, how='inner', lsuffix="_")

In [46]:
train_set_encoded.head(10)

,id_,keyword,location,text_,len_text,target_,0,00,01,02,...,zotar,zouma,zourryart,zrnf,zss,zumiez,zurich,zxatheti,zzzz,¤
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,65,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,NaN,NaN,forest fire near la rong sask canada,37,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,NaN,NaN,all resid ask to ishelt in place are be notifi...,108,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,NaN,NaN,13000 peopl receiv wildfir evacu order in cali...,53,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,NaN,NaN,just got sent thi photo from rubi alaska as sm...,82,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,8,NaN,NaN,rockyfir updat california hwi 20 close in both...,92,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,10,NaN,NaN,flood disast heavi rain cau flash flood of str...,80,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,13,NaN,NaN,i m on top of the hill and i can see a fire in...,56,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,14,NaN,NaN,there i an emerg evacu happen now in the build...,65,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,15,NaN,NaN,i m afraid that the tornado is come to our area,48,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Random Forest

In [47]:
from sklearn.model_selection import train_test_split

col_names.append('id_')
col_names.append('len_text')
col_names.append('target_')

train_set = train_set_encoded[col_names]

train_set.head(10)

,0,00,01,02,0215,03,030,034,039,05,...,zrnf,zss,zumiez,zurich,zxatheti,zzzz,¤,id_,len_text,target_
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,65,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,37,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,108,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,53,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,82,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,92,1
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,80,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,56,1
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,65,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,48,1


In [48]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [50]:
rf_model = RandomForestClassifier(n_estimators=100, min_samples_split=20)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.884781


### Test

In [46]:
test_set = pd.read_csv('test.csv')

In [47]:
test_set.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [48]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [49]:
pos_col_text = test_set.columns.get_loc('text')+1
test_set.insert(loc=pos_col_text, column='len_text', value=test_set['text'].iloc[:].str.len())
test_set.head(10)

,id,keyword,location,text,len_text
0,0,NaN,NaN,Just happened a terrible car crash,34
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45
5,12,NaN,NaN,We're shaking...It's an earthquake,34
6,21,NaN,NaN,They'd probably still show more life than Arse...,72
7,22,NaN,NaN,Hey! How are you?,17
8,27,NaN,NaN,What a nice hat?,16
9,29,NaN,NaN,Fuck off!,9


In [50]:
tf_idf_data = tf_idf_vec.fit_transform(test_set['text'])

In [51]:
col_names = tf_idf_vec.get_feature_names()

In [52]:
tf_idf_array = tf_idf_data.toarray()

In [53]:
tf_idf_df = pd.DataFrame(tf_idf_array, columns=col_names)
test_set_encoded = test_set.join(other=tf_idf_df, how='inner', lsuffix="_")

In [54]:
col_names.append('id_')
col_names.append('len_text')

test_set = test_set_encoded[col_names]

test_set.head(10)

,!,#,$,%,&,','','2079,'30,'79,...,å¨,åà,åáåáåá,åè,åê,åêwas,ìàekdar,ìñ2,id_,len_text
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,34
1,0.000000,0.125999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,64
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,96
3,0.000000,0.290634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,40
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,45
5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,34
6,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,72
7,0.395479,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22,17
8,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27,16
9,0.465408,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,9


In [55]:
preds = rf_model.predict(test_set)
preds

ValueError: Number of features of the model must match the input. Model n_features is 22783 and input n_features is 12623 

In [43]:
print(len(preds))
test_set['target'] = preds
test_set['target']

2284


ValueError: Length of values does not match length of index